## adv_statistics.py

In [ ]:
#still working...
#different statistics over low level features
def negentropy(A):
	ne = []
	for u in A:
		a1 = 2
		G1 = 1/a1*np.log(np.cosh(2*u))#np.cosh是双曲余弦函数，我不清楚它这个代码是什么目的
		v = np.random.normal(0, 1, len(u))#生成u长度的服从标准正态分布的随机数
		G2 = -np.exp(-np.power(v,2)/2)
		ne.append(np.power(np.mean(G1)-np.mean(G2),2))
	return np.array(ne)

#good for discrete features
def differences_entropy(A):
	dH = []
	for f in A:
		Y = np.diff(f)#后一个减去前一个
		P = []
		for yk in Y:
			ntimes = Y.tolist().count(yk)
			P.append(ntimes/len(Y))#最终P有个屁用？
		dH.append(entropy(Y)/np.log(len(Y)))
	return np.array(dH)

## dataset.py

In [ ]:
from pyAudioAnalysis import audioBasicIO
import os
import subprocess as sp
import itertools

class Dataset:

	# Dataset object is composed of:
	# data 
	# targets
	# train and test sets for cross validation
	# classes dictionary to map classes to numbers

	def __init__(self,path,decode):
        self.classes = {0:'W', 1:'L', 2:'E', 3:'A', 4:'F', 5:'T', 6:'N'}
        self.get_berlin_dataset(path)

	def get_berlin_dataset(self,path):
		males = ['03','10','11','12','15']
		females = ['08','09','13','14','16']
		classes = {v: k for k, v in self.classes.iteritems()}
		self.targets = []; self.data = []; self.train_sets = []; self.test_sets = []; get_data = True
		for speak_test in itertools.product(males,females):#test_couples:
			i = 0; train = []; test = [];
			for audio in os.listdir(path):
				audio_path = os.path.join(path,audio)
				[Fs,x] = audioBasicIO.readAudioFile(audio_path)
				if get_data:
					self.data.append((x,Fs))
					self.targets.append(classes[audio[5]])
				if audio[:2] in speak_test:
					test.append(i)
				else:
					train.append(i)
				i = i + 1
			self.train_sets.append(train)
			self.test_sets.append(test)
			get_data = False

## preprocessing.py

In [ ]:
from scipy.linalg import eigh
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_classif

class Eigenspectrum:

	# Eigenspectrum is composed of:
	# eigenvalues
	# % info for each eigenvalue
	# you can plot it using show()

	def __init__(self, F):
		self.eigenvalues = eigh(np.cov(F.T),eigvals_only=True)
		total_inf = np.sum(self.eigenvalues)
		self.info = self.eigenvalues/total_inf*100

	def show(self):
		info = self.info[::-1]
		plt.plot(range(0,len(self.eigenvalues)),info)
		plt.title('Eigenspectrum')
		plt.show()

class Preprocessor:

	# Preprocessor is composed of:
	# scaler (minmax or standard)
	# a pc projector
	# you can also remove features using mutual_info_select()

	def __init__(self,scaler_type,n_components):
		if scaler_type == "standard":
			self.scaler = StandardScaler()
		elif scaler_type == "minmax":
			self.scaler = MinMaxScaler()
		self.pca = PCA(n_components)

	def standardize(self,Ftrain,Ftest):
		Ftrain_std = self.scaler.fit_transform(Ftrain)
		Ftest_std = self.scaler.transform(Ftest)
		return (Ftrain_std,Ftest_std)

	def project_on_pc(self,Ftrain,Ftest):
		Ftrain_pca = self.pca.fit_transform(Ftrain)
		Ftest_pca = self.pca.transform(Ftest)
		return (Ftrain_pca,Ftest_pca)

	def mutual_info_select(self,F,y,threshold):
		mi = list(enumerate(mutual_info_classif(F,y)))
		f_best = []
		for (ind,rank) in mi:
			if rank > threshold:
				f_best.append(ind)
		return f_best

## emorecognition.py

In [ ]:
from pyAudioAnalysis import audioFeatureExtraction
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support
from sklearn import svm
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn.multiclass import OneVsRestClassifier
from argparse import ArgumentParser
import numpy as np
from scipy import spatial
from scipy.stats import entropy
import scipy.io as sio
import numpy as np
import cPickle
from time import sleep
import sys
from dataset import Dataset
from preprocessing import Eigenspectrum, Preprocessor
#from adv_statistics import negentropy, differences_entropy

if __name__ == '__main__':
	#warnings.filterwarnings('ignore')
	parser = ArgumentParser()
	parser.add_augument("-d", "--dataset", dest="db_type", default="berlin")
	parser.add_augument("-p", "--dataset_path", dest="path", default="")
	parser.add_augument("-l", "--load_data", action="store_true", dest="load_data")
	parser.add_augument("-e", "--extract_features", action="store_true", dest="extract_features")
	parser.add_augument("-s", "--speaker_indipendence", action="store_true", dest="speaker_indipendence")
	parser.add_augument("-i", "--plot_eigenspectrum", action="store_true", dest="plot_eigenspectrum")
	(options, args) = parser.parse_args(sys.argv)
	load_data = options.load_data
	extract_features = options.extract_features
	db_type = options.db_type
	speaker_indipendence = options.speaker_indipendence
	path = options.path
	plot_eigenspectrum = options.plot_eigenspectrum

	if load_data:
		print("Loading data from " + db_type + " dataset...")
		if db_type not in ('berlin'):
			sys.exit("Dataset not registered. Please create a method to read it")

		db = Dataset(path,decode=False)

		print("Saving " + db_type + " dataset info to file...")
		cPickle.dump(db, open(db_type + '_db.p', 'wb')) 
	else:
		print "Getting data from " + db_type + " dataset..."
		db = cPickle.load(open(db_type + '_db.p', 'rb'))

	n_samples = len(db.targets)
	print "Number of dataset samples: " + str(n_samples)

	if extract_features:
		win_size = 0.04
		step = 0.01
		Fglobal = []
		i = 0
		for (x,Fs) in db.data:
			F = audioFeatureExtraction.stFeatureExtraction(x, Fs, win_size*Fs, step*Fs)
			Fglobal.append( np.concatenate((	np.mean(F, axis=1),
												np.std(F, axis=1))))

			sys.stdout.write("\033[F") # cursor up one line
			i = i+1; print "Extracting features " + str(i) + '/' + str(n_samples) + " from data..."

		print "Saving features to file..."
		cPickle.dump(Fglobal, open(db_type + '_features.p', 'wb')) 
	else:
		print "Getting features from files..."
		Fglobal = cPickle.load(open(db_type + '_features.p', 'rb'))

	Fglobal = np.array(Fglobal)
	y = np.array(db.targets)

	# evaluating SVM using cross validation
	print "Evaluating model with cross validation..."

	if speaker_indipendence:
		k_folds = len(db.test_sets)
		splits = zip(db.train_sets,db.test_sets)
	else:
		k_folds = 10
		sss = StratifiedShuffleSplit(n_splits=k_folds, test_size=0.2, random_state=1)
		splits = sss.split(Fglobal, y)

	# setting preprocessing
	pp = Preprocessor('standard',n_components=50)
	n_classes = len(db.classes)
	clf = OneVsRestClassifier(svm.SVC(kernel='rbf',C=10, gamma=0.01))
	prfs = []; scores = []; acc = np.zeros(n_classes)
	mi_threshold = 0.0
	for (train,test) in splits:
		# selecting features using mutual information
		Ftrain = Fglobal[train]; Ftest = Fglobal[test]
		f_subset = pp.mutual_info_select(Ftrain,y[train],mi_threshold)
		Ftrain = Ftrain[:,f_subset]; Ftest = Ftest[:,f_subset]
		
		#standard transformation
		(Ftrain,Ftest) = pp.standardize(Ftrain,Ftest)
		
		# eigenspectrum over all data
		if plot_eigenspectrum:
			es = Eigenspectrum(Ftrain)
			es.show()
		
		(Ftrain,Ftest) = pp.project_on_pc(Ftrain,Ftest)

		clf.fit(Ftrain, y[train])
		ypred = clf.predict(Ftest)

		#print clf.score(Ftest, y[test])
		scores.append(clf.score(Ftest, y[test]))
		#print precision_recall_fscore_support(y[test], ypred)
		prfs.append(precision_recall_fscore_support(y[test], ypred))

	# mean total accuracy
	print("\nAccuracy =  %0.2f (%0.2f)\n" % (np.mean(scores), np.std(scores)))

	#mean per class precision and recall 
	mean_prec = np.zeros((1,n_classes))
	mean_recall = np.zeros((1,n_classes))
	precs = []; recalls = []
	for mat in prfs:
		precs.append(mat[0])
		recalls.append(mat[1])
		mean_prec = mean_prec + mat[0]
		mean_recall = mean_recall + mat[1]
	mean_prec = mean_prec[0] / k_folds
	mean_recall = mean_recall[0] / k_folds

	#mean total recall and precision
	precs = np.array(precs)
	recalls = np.array(recalls)
	prec_mean = np.mean(precs,axis=0)
	prec_std = np.std(precs,axis=0)
	recall_mean = np.mean(recalls,axis=0)
	recall_std = np.std(recalls,axis=0)
	print "Recall %0.2f (%0.2f)" % (np.mean(recall_mean), np.std(recall_mean))
	print "Precision: %0.2f (%0.2f)\n" % (np.mean(prec_mean), np.std(prec_mean))

	for i in range(0,n_classes):
		print db.classes[i] + " precision = %0.2f (%0.2f)" % (prec_mean[i],prec_std[i])
		print db.classes[i] + " recall = %0.2f (%0.2f)\n" % (recall_mean[i],recall_std[i])
